In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import sqlite3
import sqlalchemy
from pandasql import sqldf

In [2]:
pysqldf = lambda q: sqldf(q, globals())

In [30]:
sqladb=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
                   names=['Age','workclass','fnlwgt','education','education_num','marital_status','occupation','relationship',
                          'race','sex','capital_gain','capital_loss','hours_per_week','native_country','range_amount'])

In [4]:
#establish a connection

In [5]:
#1.Create an sqlalchemy engine using a sample from the data set

In [7]:
from sqlalchemy import create_engine

In [132]:
engine = create_engine('sqlite:///:memory:', echo = False)

In [78]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [79]:
from sqlalchemy import Column, Integer, String

In [31]:
sqladb.head(2)

,Age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,range_amount
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K


In [138]:
class User(Base):
    __tablename__ = 'data_table'
    
    ID = Column(Integer, primary_key=True)
    Age = Column(Integer)
    workclass = Column(String)
    fnlwgt = Column(Integer)
    education = Column(String)
    marital_status = Column(String)

def __repr__(self):
    return "<data(Age='%d', workcalss='%s', fnlwgt='%d', education = '%s', marital_stauts = '%s')>" % (self.Age, self.workclass, self.fnlwgt,self.education,self.marital_status)

In [139]:
User.__table__

Table('data_table', MetaData(bind=None), Column('ID', Integer(), table=<data_table>, primary_key=True, nullable=False), Column('Age', Integer(), table=<data_table>), Column('workclass', String(), table=<data_table>), Column('fnlwgt', Integer(), table=<data_table>), Column('education', String(), table=<data_table>), Column('marital_status', String(), table=<data_table>), schema=None)

In [140]:
Base.metadata.create_all(engine)

In [141]:
user = User()

In [142]:
new_user = User (  Age = '30',workclass = 'State_gov', fnlwgt = '70',education = 'Bachelors',marital_status = 'married')

In [143]:
new_user.workclass

'State_gov'

In [144]:
new_user2 = User (  Age = '32',workclass = 'Private', fnlwgt = '73',education = 'Masters',marital_status = 'married')

In [145]:
from sqlalchemy.orm import sessionmaker

In [146]:
Session = sessionmaker(bind = engine)
Session.configure(bind = engine)

In [147]:
session = Session()

In [148]:
session.add(new_user)
session.add(new_user2)

In [65]:
# 2.Write two basic update queries

In [218]:
session.add_all([
     User(Age = '35',workclass = 'State_gov', fnlwgt = '70', education = 'Masters', marital_status = 'married'),
    User(Age = '36',workclass = 'Private', fnlwgt = '75', education = 'Bachelors', marital_status = 'single',)])

In [219]:
session.query(User).all()

In [159]:
from sqlalchemy import update

In [220]:
update1 = session.query(User).filter_by(marital_status = 'single').first()

In [221]:
update1.marital_status

'single'

In [222]:
update1.marital_status = 'married'
session.commit()

In [223]:
update1.marital_status

'married'

In [233]:
update2 = session.query(User).filter_by(education = 'Bachelors').first()

In [236]:
update2.education = 'Masters'
session.commit()

In [237]:
update2.education

'Masters'

In [238]:
#3. Write two delete queries

In [239]:
from sqlalchemy import delete

In [241]:
delete1 = session.query(User).filter_by(Age = '30').first()

In [250]:
session.delete(delete1)
session.commit()

In [252]:
session.query(User).all()
#row has been deleted as only 5 rows are in the table

In [253]:
delete2 = session.query(User).filter_by(Age = '36').first()

In [254]:
session.delete(delete2)
session.commit()

In [270]:
session.query(User).all()
#row has been deleted as only 4 rows are in the table
for instance in session.query(User):
    print(instance.workclass, instance.fnlwgt, instance.education, instance.marital_status)

Private 73 Masters married
State_gov 70 Masters married
State_gov 70 Masters married
Private 75 Bachelors married


In [256]:
#4. Write two filter queries

In [281]:
for u in session.query(User).filter(User.Age =='32').filter(User.education=='Masters'):
    print(u.Age,u.workclass, u.fnlwgt, u.education, u.marital_status)

32 Private 73 Masters married


In [283]:
for u in session.query(User).filter(User.workclass =='Private').filter(User.education=='Bachelors'):
    print(u.Age,u.workclass, u.fnlwgt, u.education, u.marital_status)

36 Private 75 Bachelors married


In [272]:
#5. Write two fucntion queries

In [284]:
from sqlalchemy import func

In [294]:
#count of people working in different workclass and education backgorund
session.query(func.count(User.workclass),User.workclass,User.education).group_by(User.workclass,User.education).all()

[(1, 'Private', 'Bachelors'),
 (1, 'Private', 'Masters'),
 (2, 'State_gov', 'Masters')]

In [296]:
#Average age in each workclass
session.query(func.avg(User.Age),User.workclass).group_by(User.workclass).all()

[(34.0, 'Private'), (35.0, 'State_gov')]